## Imports

In [1]:
#importing necessary libraries
from rajvi_mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

## Importing Coverage Function (allows me to pick any branch I want)

In [234]:
#reading in library branch and population demographics, both with geometry data
#branchInfo = pd.read_csv('../library neighborhoods/completedNeighborhoodRadiiDF.csv')
#branchInfo = pd.read_csv('../library neighborhoods/5branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/7minus5branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/9minus7branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/11minus9branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/13minus11branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/15minus13branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/17minus15branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/19minus17branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/21minus19branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/23minus21branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/25minus23branchRadii.csv')
#branchInfo = pd.read_csv('../library neighborhoods/27minus25branchRadii.csv')
branchInfo = pd.read_csv('../library neighborhoods/29minus27branchRadii.csv')
censusTracts = pd.read_csv('../data/clean/population_demographics.csv')
censusDemographics = pd.read_csv('../data/dani data/census_demos.csv')

In [235]:
#removing unecessary columns from branchInfo
branchInfo = branchInfo.drop(columns = ['Unnamed: 0'])
#removing unecessary columns from censusTracts
censusTracts = censusTracts[['geoid','geometry','qualifying name']]
#converting censusTracts to a geodataframe (we could not load it as one since it already had a geometry column so we are simply
#converting the geometry column to a proper gpd geometry column).
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )
branchInfo[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchInfo['LOCATION']]
branchInfo.loc[:, 'LATITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LATITUDE'])
branchInfo.loc[:, 'LONGITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LONGITUDE'])
branchInfo = gpd.GeoDataFrame(
    branchInfo.loc[:, [c for c in branchInfo.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(branchInfo["geometry"]),
    crs="epsg:4326",
    )

In [178]:
censusPops = censusDemographics[['geoid', 'total population', 'Percent: Black or African American Alone']].copy(deep=True)
censusPops['Black Pop'] = censusPops['total population']*censusPops['Percent: Black or African American Alone']
censusPops = censusPops.drop(columns = ['total population', 'Percent: Black or African American Alone'])
censusPops.head(3)

,geoid,Black Pop
0,17031823303,317.0
1,17031823304,1923.0
2,17031740400,275.0


In [167]:
#helper method to ensure that two polygons overlap before getting the intersection. Gets the intersection then calculates
#the percent of overlapping area for a census tract and saves it to a dictionary with the census tract geo-id as the key and
#the percent as the value.
def check(polygon1, df, geoname, dictionary,key):
   
    for i in range(len(df)):
        if polygon1.intersects(df.loc[i,geoname]): 
            overlapPolygon = (polygon1.intersection(df.loc[i,geoname]))
            poly_area, poly_perimeter = geod.geometry_area_perimeter(overlapPolygon)
            overlapArea = poly_area*-1
            propOverlap = overlapArea / df.loc[i,'Area']
            #print(propOverlap)
            dictionary[df.loc[i, key]] = propOverlap
    return dictionary
   

In [168]:
#sets the crs of gpd types
def geoSetup(gdf):
    
    gdf = gdf.set_crs('EPSG:4326')
    

In [169]:
#cdf = gdf of geometry area which you want to get the coverage score
#idf = gdf of points for which you want to get an isochrone and dictionary of percent of overlap in cdf areas
#scoreString = a string that is the name of the coverage score column you choose
#dictString = a string that is the name of the dictionary column you choose
#cgs = a string that is the name of the geometry column in the cdf
#igs = a string that is the name of the geometry column in the idf
#lat = a string that is the name of the latitude column in the idf
#lon = a string that is the name of the longitude column in the idf
#key = a string that is the name of the column in the cdf that you want to represent the key of the dictionary

def appendADS(cdf,idf,scoreString,dictString,cgs,igs,lat,lon,key):
    
    #api = MapboxAPI()
    cdf[scoreString] = 0.0
    idf[dictString] = ''
    censusAreas = []
    global geod 
    geod = Geod(ellps='WGS84')
   
    geoSetup(cdf)
    geoSetup(idf)
   
    for index, row in cdf.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row[cgs])
        poly_area = poly_area*-1
        censusAreas.append(poly_area)
    
    cdf['Area'] = censusAreas



    for i in range(len(idf)):
        dictionary = {}
        dictionary = check(idf.loc[i,igs],cdf,cgs, dictionary,key)
        branchInfo.loc[i,dictString]= [dictionary]
    
        for i in range(len(cdf)):
   
            if dictionary.get(cdf.loc[i, key]) != None:
                score = cdf.loc[i, scoreString]
                cdf.loc[i, scoreString] = score + dictionary.get(cdf.loc[i, key])
  
   

In [236]:
appendADS(censusTracts,branchInfo, 'score', 'list of dict','geometry','geometry','LATITUDE','LONGITUDE','geoid')

/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/tljh/user/lib/python3.9/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


In [171]:
branchInfo.head(3)

,BRANCH,LOCATION,LATITUDE,LONGITUDE,geometry,list of dict
0,Albany Park,"(41.97557881655979, -87.71361314512697)",41.975579,-87.713613,"POLYGON ((-87.71361 41.97980, -87.71561 41.978...","[{17031140200: 0.024261414648178242, 170318318..."
1,Altgeld,"(41.65719847558056, -87.59883829075173)",41.657198,-87.598838,"POLYGON ((-87.59884 41.65834, -87.60070 41.658...","[{17031540101: 0.03491984488141061, 1703154010..."
2,Archer Heights,"(41.80110836194246, -87.72648385568911)",41.801108,-87.726484,"POLYGON ((-87.72448 41.80420, -87.72548 41.804...","[{17031620200: 0.02353879749607556, 1703157050..."


## Average Travel Time Function

In [90]:

#Internet
def min5Radius(DemographicsVariables, branchWithCensus):
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    #Branch_pop_withInternet = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['Total']).sum() 
    #result = Branch_pop_withInternet/pop_in_branch
    return pop_in_branch   

#Income


In [91]:
def branch_demographics(branchWithCensus, nameOfBranch):
    branch_snapshot_df = pd.DataFrame({'Branch': [nameOfBranch]})
    list1 = ['5 Min']
    for i in list1:
        branch_snapshot_df[i] = min5Radius(i, branchWithCensus)
    
    return branch_snapshot_df

In [172]:
def incrementPopulations(branchDataset):
    popPerLib = pd.DataFrame(columns=['Branch', 'Given Min Pop'])

    for i in range(len(branchDataset)):
        
        #Locating only Altgeld's column 'list of dict'
        libraryDictList = branchDataset.loc[i,'list of dict']
        branchName = branchDataset.loc[i,'BRANCH']
        
        #Removed set of brackets, just a dictionary now
        finalDictionary = libraryDictList[0]

        #Seperating into two columns
        dictToList = pd.DataFrame(finalDictionary.items(), columns= ['geoid','Percent Overlap'])
        libraryAndCensus = censusPops.merge(dictToList, on='geoid')

        libraryAndCensus['counts'] = 0
        for j in range(len(libraryAndCensus)):
            libraryAndCensus.loc[j,'counts'] = libraryAndCensus.loc[j,'Black Pop'] * libraryAndCensus.loc[j,'Percent Overlap']
        
        libPop = pd.DataFrame({'BRANCH': [branchName],'Given Min Pop': libraryAndCensus['counts'].sum()})
        popPerLib = pd.concat([popPerLib, libPop])
        
    popPerLib = popPerLib.drop(columns = ['Branch'])
    popPerLib = popPerLib.reset_index()
    popPerLib = popPerLib.drop(columns = ['index'])
    
    return popPerLib

In [237]:
oneRing = incrementPopulations(branchInfo)

In [238]:
c29Min27Population = oneRing.copy(deep = True)
c29Min27Population

,Given Min Pop,BRANCH
0,8.563002,Albany Park
1,644.423453,Altgeld
2,114.939563,Archer Heights
3,872.886415,Austin
4,0.701763,Austin-Irving
...,...,...
76,1489.846950,West Pullman
77,0.000000,West Town
78,1806.048718,"Whitney M. Young, Jr."
79,1586.069144,Woodson Regional Library


In [239]:
allRadiiPops = pd.DataFrame(columns=['Branch', 'Prop 0 to 5 Min', 'Prop 5 to 7 Min', 'Prop 7 to 9 Min', 'Prop 9 to 11 Min', 
                                     'Prop 11 to 13 Min', 'Prop 13 to 15 Min', 'Prop 15 to 17 Min', 'Prop 17 to 19 Min',
                                     'Prop 19 to 21 Min', 'Prop 21 to 23 Min', 'Prop 23 to 25 Min', 'Prop 25 to 27 Min', 
                                     'Prop 27 to 29 Min', 'Total Pop Around Library'])

In [240]:
allRadiiPops['Branch'] = c5Min0Population['BRANCH']
allRadiiPops['Prop 0 to 5 Min'] = c5Min0Population['Given Min Pop']
allRadiiPops['Prop 5 to 7 Min'] = c7Min5Population['Given Min Pop']
allRadiiPops['Prop 7 to 9 Min'] = c9Min7Population['Given Min Pop']
allRadiiPops['Prop 9 to 11 Min'] = c11Min9Population['Given Min Pop']
allRadiiPops['Prop 11 to 13 Min'] = c13Min11Population['Given Min Pop']
allRadiiPops['Prop 13 to 15 Min'] = c15Min13Population['Given Min Pop']
allRadiiPops['Prop 15 to 17 Min'] = c17Min15Population['Given Min Pop']
allRadiiPops['Prop 17 to 19 Min'] = c19Min17Population['Given Min Pop']
allRadiiPops['Prop 19 to 21 Min'] = c21Min19Population['Given Min Pop']
allRadiiPops['Prop 21 to 23 Min'] = c23Min21Population['Given Min Pop']
allRadiiPops['Prop 23 to 25 Min'] = c25Min23Population['Given Min Pop']
allRadiiPops['Prop 25 to 27 Min'] = c27Min25Population['Given Min Pop']
allRadiiPops['Prop 27 to 29 Min'] = c29Min27Population['Given Min Pop']

In [241]:
allRadiiPops

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library
0,Albany Park,69.191246,129.275271,199.179278,258.433754,287.768052,285.984424,195.052760,94.215789,38.412584,52.849118,9.609511,7.710776,8.563002,NaN
1,Altgeld,174.739384,195.747004,208.894317,254.224009,257.350078,279.094759,300.644604,271.910530,181.687348,274.726166,390.532055,538.565974,644.423453,NaN
2,Archer Heights,29.139278,18.088037,23.396261,19.219654,15.571212,32.377238,42.959679,38.792646,44.005486,49.087170,71.288447,96.641866,114.939563,NaN
3,Austin,2289.952743,2250.542901,3012.893168,3682.921653,3516.187635,3676.995880,3095.175535,1984.565614,1529.443613,1632.170288,1746.462002,946.210068,872.886415,NaN
4,Austin-Irving,41.717411,40.114706,56.469033,67.738101,87.526912,76.670199,76.762373,65.459883,61.767796,21.082068,21.281421,2.870703,0.701763,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,1067.889271,967.718326,1170.446301,1684.434827,1949.677012,2259.070863,2800.075932,2781.690683,3222.556499,3878.137085,4687.824896,5956.562617,1489.846950,NaN
77,West Town,273.293447,246.886852,300.442774,323.433271,418.891999,452.449812,167.048851,188.301557,149.942114,47.163891,26.316869,2.637586,0.000000,NaN
78,"Whitney M. Young, Jr.",2581.819958,2139.508883,2738.937490,2863.670738,3292.229700,3703.871573,3202.463754,2532.543890,1994.082771,2024.802990,1990.861175,1637.315585,1806.048718,NaN
79,Woodson Regional Library,1361.284068,1258.594257,1620.539420,1957.737387,2278.847235,2743.838860,2437.750833,2347.343605,2272.633486,2092.342720,2340.212811,2264.895510,1586.069144,NaN


In [242]:
allRadiiPops = allRadiiPops.fillna(0)

In [243]:
allRadiiPops = allRadiiPops.set_index('Branch')

In [244]:
allRadiiPops['Total Pop Around Library'] = allRadiiPops.sum(axis=1)

In [245]:
allRadiiPops

,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library
Branch,,,,,,,,,,,,,,
Albany Park,69.191246,129.275271,199.179278,258.433754,287.768052,285.984424,195.052760,94.215789,38.412584,52.849118,9.609511,7.710776,8.563002,1636.245565
Altgeld,174.739384,195.747004,208.894317,254.224009,257.350078,279.094759,300.644604,271.910530,181.687348,274.726166,390.532055,538.565974,644.423453,3972.539683
Archer Heights,29.139278,18.088037,23.396261,19.219654,15.571212,32.377238,42.959679,38.792646,44.005486,49.087170,71.288447,96.641866,114.939563,595.506536
Austin,2289.952743,2250.542901,3012.893168,3682.921653,3516.187635,3676.995880,3095.175535,1984.565614,1529.443613,1632.170288,1746.462002,946.210068,872.886415,30236.407515
Austin-Irving,41.717411,40.114706,56.469033,67.738101,87.526912,76.670199,76.762373,65.459883,61.767796,21.082068,21.281421,2.870703,0.701763,620.162371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Pullman,1067.889271,967.718326,1170.446301,1684.434827,1949.677012,2259.070863,2800.075932,2781.690683,3222.556499,3878.137085,4687.824896,5956.562617,1489.846950,33915.931262
West Town,273.293447,246.886852,300.442774,323.433271,418.891999,452.449812,167.048851,188.301557,149.942114,47.163891,26.316869,2.637586,0.000000,2596.809023
"Whitney M. Young, Jr.",2581.819958,2139.508883,2738.937490,2863.670738,3292.229700,3703.871573,3202.463754,2532.543890,1994.082771,2024.802990,1990.861175,1637.315585,1806.048718,32508.157225


In [246]:
allRadiiPops.reset_index(inplace = True)

In [247]:
allRadiiPops

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library
0,Albany Park,69.191246,129.275271,199.179278,258.433754,287.768052,285.984424,195.052760,94.215789,38.412584,52.849118,9.609511,7.710776,8.563002,1636.245565
1,Altgeld,174.739384,195.747004,208.894317,254.224009,257.350078,279.094759,300.644604,271.910530,181.687348,274.726166,390.532055,538.565974,644.423453,3972.539683
2,Archer Heights,29.139278,18.088037,23.396261,19.219654,15.571212,32.377238,42.959679,38.792646,44.005486,49.087170,71.288447,96.641866,114.939563,595.506536
3,Austin,2289.952743,2250.542901,3012.893168,3682.921653,3516.187635,3676.995880,3095.175535,1984.565614,1529.443613,1632.170288,1746.462002,946.210068,872.886415,30236.407515
4,Austin-Irving,41.717411,40.114706,56.469033,67.738101,87.526912,76.670199,76.762373,65.459883,61.767796,21.082068,21.281421,2.870703,0.701763,620.162371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,1067.889271,967.718326,1170.446301,1684.434827,1949.677012,2259.070863,2800.075932,2781.690683,3222.556499,3878.137085,4687.824896,5956.562617,1489.846950,33915.931262
77,West Town,273.293447,246.886852,300.442774,323.433271,418.891999,452.449812,167.048851,188.301557,149.942114,47.163891,26.316869,2.637586,0.000000,2596.809023
78,"Whitney M. Young, Jr.",2581.819958,2139.508883,2738.937490,2863.670738,3292.229700,3703.871573,3202.463754,2532.543890,1994.082771,2024.802990,1990.861175,1637.315585,1806.048718,32508.157225
79,Woodson Regional Library,1361.284068,1258.594257,1620.539420,1957.737387,2278.847235,2743.838860,2437.750833,2347.343605,2272.633486,2092.342720,2340.212811,2264.895510,1586.069144,26562.089335


In [248]:
allRadiiPops['Prop 0 to 5 Min'] = allRadiiPops['Prop 0 to 5 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 5 to 7 Min'] = allRadiiPops['Prop 5 to 7 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 7 to 9 Min'] = allRadiiPops['Prop 7 to 9 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 9 to 11 Min'] = allRadiiPops['Prop 9 to 11 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 11 to 13 Min'] = allRadiiPops['Prop 11 to 13 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 13 to 15 Min'] = allRadiiPops['Prop 13 to 15 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 15 to 17 Min'] = allRadiiPops['Prop 15 to 17 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 17 to 19 Min'] = allRadiiPops['Prop 17 to 19 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 19 to 21 Min'] = allRadiiPops['Prop 19 to 21 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 21 to 23 Min'] = allRadiiPops['Prop 21 to 23 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 23 to 25 Min'] = allRadiiPops['Prop 23 to 25 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 25 to 27 Min'] = allRadiiPops['Prop 25 to 27 Min']/allRadiiPops['Total Pop Around Library']
allRadiiPops['Prop 27 to 29 Min'] = allRadiiPops['Prop 27 to 29 Min']/allRadiiPops['Total Pop Around Library']

In [249]:
allRadiiPops

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library
0,Albany Park,0.042287,0.079007,0.121729,0.157943,0.175871,0.174781,0.119208,0.057580,0.023476,0.032299,0.005873,0.004712,0.005233,1636.245565
1,Altgeld,0.043987,0.049275,0.052585,0.063995,0.064782,0.070256,0.075681,0.068448,0.045736,0.069156,0.098308,0.135572,0.162220,3972.539683
2,Archer Heights,0.048932,0.030374,0.039288,0.032274,0.026148,0.054369,0.072140,0.065142,0.073896,0.082429,0.119711,0.162285,0.193011,595.506536
3,Austin,0.075735,0.074432,0.099645,0.121804,0.116290,0.121608,0.102366,0.065635,0.050583,0.053980,0.057760,0.031294,0.028869,30236.407515
4,Austin-Irving,0.067269,0.064684,0.091055,0.109226,0.141135,0.123629,0.123778,0.105553,0.099599,0.033994,0.034316,0.004629,0.001132,620.162371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.031486,0.028533,0.034510,0.049665,0.057486,0.066608,0.082559,0.082017,0.095016,0.114346,0.138219,0.175627,0.043928,33915.931262
77,West Town,0.105242,0.095073,0.115697,0.124550,0.161310,0.174233,0.064329,0.072513,0.057741,0.018162,0.010134,0.001016,0.000000,2596.809023
78,"Whitney M. Young, Jr.",0.079421,0.065815,0.084254,0.088091,0.101274,0.113937,0.098513,0.077905,0.061341,0.062286,0.061242,0.050366,0.055557,32508.157225
79,Woodson Regional Library,0.051249,0.047383,0.061009,0.073704,0.085793,0.103299,0.091776,0.088372,0.085559,0.078772,0.088103,0.085268,0.059712,26562.089335


In [250]:
allRadiiPops['Weighted Average Travel Time'] = 0

In [251]:
allRadiiPops

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.042287,0.079007,0.121729,0.157943,0.175871,0.174781,0.119208,0.057580,0.023476,0.032299,0.005873,0.004712,0.005233,1636.245565,0
1,Altgeld,0.043987,0.049275,0.052585,0.063995,0.064782,0.070256,0.075681,0.068448,0.045736,0.069156,0.098308,0.135572,0.162220,3972.539683,0
2,Archer Heights,0.048932,0.030374,0.039288,0.032274,0.026148,0.054369,0.072140,0.065142,0.073896,0.082429,0.119711,0.162285,0.193011,595.506536,0
3,Austin,0.075735,0.074432,0.099645,0.121804,0.116290,0.121608,0.102366,0.065635,0.050583,0.053980,0.057760,0.031294,0.028869,30236.407515,0
4,Austin-Irving,0.067269,0.064684,0.091055,0.109226,0.141135,0.123629,0.123778,0.105553,0.099599,0.033994,0.034316,0.004629,0.001132,620.162371,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.031486,0.028533,0.034510,0.049665,0.057486,0.066608,0.082559,0.082017,0.095016,0.114346,0.138219,0.175627,0.043928,33915.931262,0
77,West Town,0.105242,0.095073,0.115697,0.124550,0.161310,0.174233,0.064329,0.072513,0.057741,0.018162,0.010134,0.001016,0.000000,2596.809023,0
78,"Whitney M. Young, Jr.",0.079421,0.065815,0.084254,0.088091,0.101274,0.113937,0.098513,0.077905,0.061341,0.062286,0.061242,0.050366,0.055557,32508.157225,0
79,Woodson Regional Library,0.051249,0.047383,0.061009,0.073704,0.085793,0.103299,0.091776,0.088372,0.085559,0.078772,0.088103,0.085268,0.059712,26562.089335,0


In [252]:
for i in range(len(allRadiiPops)):
    allRadiiPops.loc[i,'Weighted Average Travel Time'] = (((allRadiiPops.loc[i,'Prop 0 to 5 Min'])*5) +
                                                        ((allRadiiPops.loc[i,'Prop 5 to 7 Min'])*6) +
                                                        ((allRadiiPops.loc[i,'Prop 7 to 9 Min'])*8) +
                                                        ((allRadiiPops.loc[i,'Prop 9 to 11 Min'])*10) +
                                                        ((allRadiiPops.loc[i,'Prop 11 to 13 Min'])*12) +
                                                        ((allRadiiPops.loc[i,'Prop 13 to 15 Min'])*14) +
                                                        ((allRadiiPops.loc[i,'Prop 15 to 17 Min'])*16) +
                                                        ((allRadiiPops.loc[i,'Prop 17 to 19 Min'])*18) +
                                                        ((allRadiiPops.loc[i,'Prop 19 to 21 Min'])*20) +
                                                        ((allRadiiPops.loc[i,'Prop 21 to 23 Min'])*22) +
                                                        ((allRadiiPops.loc[i,'Prop 23 to 25 Min'])*24) +
                                                        ((allRadiiPops.loc[i,'Prop 25 to 27 Min'])*26) +
                                                         (allRadiiPops.loc[i,'Prop 27 to 29 Min'])*28)

In [253]:
allRadiiPops

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.042287,0.079007,0.121729,0.157943,0.175871,0.174781,0.119208,0.057580,0.023476,0.032299,0.005873,0.004712,0.005233,1636.245565,12.330002
1,Altgeld,0.043987,0.049275,0.052585,0.063995,0.064782,0.070256,0.075681,0.068448,0.045736,0.069156,0.098308,0.135572,0.162220,3972.539683,18.642701
2,Archer Heights,0.048932,0.030374,0.039288,0.032274,0.026148,0.054369,0.072140,0.065142,0.073896,0.082429,0.119711,0.162285,0.193011,595.506536,20.253843
3,Austin,0.075735,0.074432,0.099645,0.121804,0.116290,0.121608,0.102366,0.065635,0.050583,0.053980,0.057760,0.031294,0.028869,30236.407515,13.965169
4,Austin-Irving,0.067269,0.064684,0.091055,0.109226,0.141135,0.123629,0.123778,0.105553,0.099599,0.033994,0.034316,0.004629,0.001132,620.162371,13.565469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.031486,0.028533,0.034510,0.049665,0.057486,0.066608,0.082559,0.082017,0.095016,0.114346,0.138219,0.175627,0.043928,33915.931262,19.050420
77,West Town,0.105242,0.095073,0.115697,0.124550,0.161310,0.174233,0.064329,0.072513,0.057741,0.018162,0.010134,0.001016,0.000000,2596.809023,11.801216
78,"Whitney M. Young, Jr.",0.079421,0.065815,0.084254,0.088091,0.101274,0.113937,0.098513,0.077905,0.061341,0.062286,0.061242,0.050366,0.055557,32508.157225,15.067852
79,Woodson Regional Library,0.051249,0.047383,0.061009,0.073704,0.085793,0.103299,0.091776,0.088372,0.085559,0.078772,0.088103,0.085268,0.059712,26562.089335,16.748017


In [254]:
allRadiiPops.sort_values(by=['Weighted Average Travel Time'], ascending=True)

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library,Weighted Average Travel Time
42,Little Italy,0.365638,0.116511,0.079672,0.103126,0.106236,0.077863,0.041455,0.038064,0.039124,0.019107,0.010640,0.002250,0.000312,3628.650158,9.434712
51,Near North,0.161264,0.184579,0.185448,0.128582,0.102176,0.056096,0.048919,0.047053,0.039935,0.030318,0.008871,0.004175,0.002585,5343.583544,10.183826
8,Bezazian,0.153494,0.143688,0.144066,0.177944,0.134072,0.102504,0.052497,0.052818,0.027943,0.007948,0.002626,0.000363,0.000036,6413.240452,10.203379
25,Edgewater,0.168435,0.164769,0.130990,0.133648,0.122064,0.118389,0.065124,0.040930,0.027091,0.023980,0.002890,0.001096,0.000593,7165.785145,10.299982
15,Chicago Bee,0.190047,0.107073,0.113608,0.115706,0.131351,0.149576,0.110811,0.080555,0.001274,0.000000,0.000000,0.000000,0.000000,6907.209571,10.577300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,Archer Heights,0.048932,0.030374,0.039288,0.032274,0.026148,0.054369,0.072140,0.065142,0.073896,0.082429,0.119711,0.162285,0.193011,595.506536,20.253843
28,Garfield Ridge,0.013767,0.014890,0.016697,0.025100,0.047986,0.063900,0.061007,0.074325,0.104780,0.170928,0.271504,0.025087,0.110028,144.841913,20.432304
18,Clearing,0.022512,0.018141,0.022525,0.028867,0.026374,0.027951,0.022145,0.046386,0.367626,0.013233,0.052651,0.237694,0.113895,1695.980141,20.863726
55,Oriole Park,0.000993,0.003258,0.004278,0.014664,0.072015,0.152411,0.053949,0.065170,0.049392,0.041231,0.122353,0.186894,0.233392,382.721828,21.465182


In [255]:
allRadiiPops

,Branch,Prop 0 to 5 Min,Prop 5 to 7 Min,Prop 7 to 9 Min,Prop 9 to 11 Min,Prop 11 to 13 Min,Prop 13 to 15 Min,Prop 15 to 17 Min,Prop 17 to 19 Min,Prop 19 to 21 Min,Prop 21 to 23 Min,Prop 23 to 25 Min,Prop 25 to 27 Min,Prop 27 to 29 Min,Total Pop Around Library,Weighted Average Travel Time
0,Albany Park,0.042287,0.079007,0.121729,0.157943,0.175871,0.174781,0.119208,0.057580,0.023476,0.032299,0.005873,0.004712,0.005233,1636.245565,12.330002
1,Altgeld,0.043987,0.049275,0.052585,0.063995,0.064782,0.070256,0.075681,0.068448,0.045736,0.069156,0.098308,0.135572,0.162220,3972.539683,18.642701
2,Archer Heights,0.048932,0.030374,0.039288,0.032274,0.026148,0.054369,0.072140,0.065142,0.073896,0.082429,0.119711,0.162285,0.193011,595.506536,20.253843
3,Austin,0.075735,0.074432,0.099645,0.121804,0.116290,0.121608,0.102366,0.065635,0.050583,0.053980,0.057760,0.031294,0.028869,30236.407515,13.965169
4,Austin-Irving,0.067269,0.064684,0.091055,0.109226,0.141135,0.123629,0.123778,0.105553,0.099599,0.033994,0.034316,0.004629,0.001132,620.162371,13.565469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,West Pullman,0.031486,0.028533,0.034510,0.049665,0.057486,0.066608,0.082559,0.082017,0.095016,0.114346,0.138219,0.175627,0.043928,33915.931262,19.050420
77,West Town,0.105242,0.095073,0.115697,0.124550,0.161310,0.174233,0.064329,0.072513,0.057741,0.018162,0.010134,0.001016,0.000000,2596.809023,11.801216
78,"Whitney M. Young, Jr.",0.079421,0.065815,0.084254,0.088091,0.101274,0.113937,0.098513,0.077905,0.061341,0.062286,0.061242,0.050366,0.055557,32508.157225,15.067852
79,Woodson Regional Library,0.051249,0.047383,0.061009,0.073704,0.085793,0.103299,0.091776,0.088372,0.085559,0.078772,0.088103,0.085268,0.059712,26562.089335,16.748017


In [256]:
averageWalkingTimeNeighborhoods = pd.DataFrame(columns=['Branch', 'Average Walking Time (Minutes)'])


In [257]:
averageWalkingTimeNeighborhoods['Branch'] = allRadiiPops['Branch']
averageWalkingTimeNeighborhoods['Average Walking Time (Minutes)'] = allRadiiPops['Weighted Average Travel Time']

In [258]:
averageWalkingTimeNeighborhoods

,Branch,Average Walking Time (Minutes)
0,Albany Park,12.330002
1,Altgeld,18.642701
2,Archer Heights,20.253843
3,Austin,13.965169
4,Austin-Irving,13.565469
...,...,...
76,West Pullman,19.050420
77,West Town,11.801216
78,"Whitney M. Young, Jr.",15.067852
79,Woodson Regional Library,16.748017


In [259]:
averageWalkingTimeNeighborhoods.to_csv('../library neighborhoods/neighborhood by demographic/BlackorAfricanAmericanAloneaverageWalkingTimeNeighborhoods.csv')